In [24]:
import json
with open("xlam_function_calling_60k/xlam_function_calling_60k.json") as f:
    data = json.load(f)

In [75]:
from datasets import Dataset

dataset_list = []
for item in data:
    query = item["query"]
    tool_answer = "<tool_call>\n" + "\n".join(json.dumps(a) for a in json.loads(item["answers"])) + "\n</tool_call>"
    tools_string = "\n".join(json.dumps(a) for a in json.loads(item["tools"]))
    system = 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n' + tools_string + '\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call>'
    data_item = {"prompt": [{"role": "system", "content": system}, {"role": "user", "content": query}],
                            "completion": [{"role": "assistant", "content": tool_answer}]}
    dataset_list.append(data_item)

dataset = Dataset.from_list(dataset_list).train_test_split(test_size=0.1, seed=1)

In [87]:
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from datasets import load_from_disk

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
training_args = SFTConfig(
    max_length=512,
    output_dir="/tmp",
    completion_only_loss=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=500,
    num_train_epochs=1,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    bf16=True,
    report_to="none",
    save_total_limit=2
)
trainer = SFTTrainer(
    "Qwen/Qwen2-0.5B-Instruct",
    train_dataset=load_from_disk("cached_tokenized_train"),
    eval_dataset=load_from_disk("cached_tokenized_test"),
    args=training_args,
    peft_config=lora_config
)
trainer.train()

Truncating eval dataset: 100%|██████████| 6000/6000 [00:00<00:00, 185917.73 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
100,0.127300,0.054921
200,0.058200,0.049007
300,0.050300,0.045656
400,0.045500,0.043320
500,0.045400,0.042121
600,0.044900,0.040418
700,0.048000,0.039355
800,0.040100,0.038784
900,0.046600,0.038052
1000,0.042800,0.037386


TrainOutput(global_step=1688, training_loss=0.04920499514064518, metrics={'train_runtime': 2664.9498, 'train_samples_per_second': 20.263, 'train_steps_per_second': 0.633, 'total_flos': 5.859846133650432e+16, 'train_loss': 0.04920499514064518})